In [1]:
#load libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
from datetime import datetime, timedelta
import re, os

In [2]:
raw_data = pd.read_csv("/Users/albk/Documents/Code/Real_Projects/basketball_league_webapp/data/Carleton_Canisius #_2023-07-30.csv")

In [3]:
player_event_df = raw_data.copy()

In [4]:
player_event_df['V-event'] = player_event_df['V-event'].astype(str)

In [5]:
#clean data
event_list = ['made layup','missed layup','Assist','Turnover','defensive rebound','enters the game','goes to the bench','missed 3-pt. jump shot','Foul','Steal','made free throw','missed free throw','made jump shot','made 3-pt. jump shot','missed jump shot','offensive rebound']

### Simply building a dataframe from main one and getting kinds of event with respect to their player

In [6]:
pattern = "([A-Z]+,[A-Z]+.?[A-Z]*)"
for index, row in player_event_df.iterrows():
    player = re.search(pattern, row["V-event"])
    if player:
        player = player[0].strip()
        player_event_df.loc[index, "player"] = player
    
    for event in event_list:
        if event in row["V-event"]:
            player_event_df.loc[index, "event"] = event

### Calculating in-game time of each player on each quarter and events count for each player also

In [120]:
# ------------------------------------------------------------------------------------------------------------------------------------------------------------
# dependencies
quarter_indices = list(player_event_df[player_event_df['Time'].str.contains('Quarter')].index)
quarter_indices.append(len(player_event_df) - 1)
quarter_dict = {"player":players_list, "time1":[], "score1":[], 
                "time2":[], "score2":[], "time3":[], "score3":[],
                "time4":[], "score4":[]}
v = list(quarter_dict.keys())
v.remove("player")
reorder_ls = ["lineup"] + v
lineup_quarter_dict = {key:[] for key in reorder_ls}
quarter = 1

in_lineup = []
starter_lineup = ['VREEKEN,CONNOR', 'WARNHOLTZ,AIDEN', 
                  'SERAPH,REGINALD JEAN', 'LATIFF,WAZIR', 
                  'SHEPHARD,GRANT']


not_changed_list = ["not_changed" for n in range(len(players_list))]
event_num_dict = {"player":[]}
time_dict = {"player":players_list, 
             "seconds":list(np.zeros(len(players_list))),
             "points_conceded":list(np.zeros(len(players_list))),
             "timecache":not_changed_list.copy(),
             "scorecache":not_changed_list.copy()}
             
lineup_time_dict = {k:[] for k in list(time_dict.keys())}
lineup_time_dict["lineup"] = lineup_time_dict.pop("player")
lineup_event_dict = {"lineup":[]}

for ind, row in player_event_df.iterrows():
# ------------------------------------------------------------------------------------------------------------------------------------------------------------
    # calculating in-game time of each player
    # -------------------------------------------
    # each quarter end calculations
    if ind in quarter_indices:
        if ind != 0:
            for player in in_lineup:
                player_ind = time_dict["player"].index(player)
                cached_time = time_dict["timecache"][player_ind]
                cached_score = time_dict["scorecache"][player_ind]
                if cached_time == "not_changed":
                    enter_time = datetime.strptime("10:00", "%M:%S")
                    enter_score_index = quarter_indices[quarter - 1] + 1
                    enter_score = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0])
                else:
                    enter_time = datetime.strptime(cached_time, "%M:%S")
                    enter_score = cached_score
                
                seconds = enter_time - datetime.strptime("00:00", "%M:%S")
                seconds = seconds.total_seconds()

                points = int(player_event_df.iloc[ind - 1]["Score"].split("-")[0]) - enter_score

                time_dict["seconds"][player_ind] += seconds
                time_dict["points_conceded"][player_ind] += points
            
            # ---------------------
            # lineup quarter calculations
            lineup_cached_score = lineup_time_dict["scorecache"][-1]
            if lineup_time_dict["timecache"][-1] == "not_changed":
                lineup_time_dict["seconds"] += 600
                enter_score_index = quarter_indices[quarter - 1] + 1
                enter_score = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0])
            else:
                cached_time = lineup_time_dict["timecache"][-1]
                enter_time = datetime.strptime(cached_time, "%M:%S")
                seconds = enter_time - datetime.strptime("00:00", "%M:%S")
                seconds = seconds.total_seconds()
                lineup_time_dict["seconds"][-1] += seconds
                enter_score = int(lineup_cached_score.split("-")[0])
            
            points = int(player_event_df.iloc[ind - 1]["Score"].split("-")[0]) - enter_score

            lineup_time_dict["seconds"][-1] += seconds
            lineup_time_dict["points_conceded"][-1] += points
            # ---------------------

            quarter_dict["player"] = time_dict["player"]
            quarter_dict[f"time{quarter_indices.index(ind)}"] = time_dict["seconds"]
            quarter_dict[f"score{quarter_indices.index(ind)}"] = time_dict["points_conceded"]
            for key in list(lineup_quarter_dict.keys()):
                if key != "lineup":
                    if int(key[-1]) == quarter:
                        if "time" in key:
                            lineup_quarter_dict[f"time{quarter}"].extend(lineup_time_dict["seconds"])
                        else:    
                            lineup_quarter_dict[f"score{quarter}"].extend(lineup_time_dict["points_conceded"])
                    else:
                        length = len(lineup_time_dict["lineup"])
                        zero_list = list(np.zeros(length))
                        lineup_quarter_dict[key].extend(zero_list)
                else:
                    lineup_quarter_dict[key].extend(lineup_time_dict["lineup"])

            
            quarter += 1
            time_dict["timecache"] = not_changed_list.copy()
            time_dict["scorecache"] = not_changed_list.copy()
            time_dict["seconds"] = list(np.zeros(len(players_list)))
            time_dict["points_conceded"] = list(np.zeros(len(players_list)))
            lineup_time_dict = {key:[] for key in lineup_time_dict}

        in_lineup = starter_lineup.copy()
        lineup_time_dict["lineup"].append(sorted(in_lineup.copy()))
        for key in lineup_time_dict:
            if key != "lineup":
                if "cache" in key:
                    lineup_time_dict[key].append("not_changed")
                else:
                    lineup_time_dict[key].append(0)

        for key in list(lineup_event_dict.keys()):
                if key == "lineup":
                    lineup_event_dict[key].append(sorted(in_lineup))
                else:
                    lineup_event_dict[key].append(0)
        continue
    # -------------------------------------------
    # iterating rows calculation
    if not pd.isna(row["player"]) and row["player"] != np.nan:
        player_index = time_dict["player"].index(row["player"])
        cached_time = time_dict["timecache"][player_index]
        cached_score = time_dict["scorecache"][player_index]
        if "goes to the bench" in row["event"]:
            in_lineup.remove(row["player"])
            if cached_time == "not_changed":
                enter_time = datetime.strptime("10:00", "%M:%S")
                enter_score_index = quarter_indices[quarter - 1] + 1
                enter_score = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0])
            else:
                enter_time = datetime.strptime(cached_time, "%M:%S")
                enter_score = cached_score
            
            seconds = enter_time - datetime.strptime(row["Time"], "%M:%S")
            seconds = seconds.total_seconds()

            points = int(row["Score"].split("-")[0]) - enter_score

            time_dict["seconds"][player_index] += seconds
            time_dict["points_conceded"][player_index] += points

        elif "enters the game" in row["event"]:
            in_lineup.append(row["player"])
            time_dict["timecache"][player_index] = row["Time"]
            time_dict["scorecache"][player_index] = int(row["Score"].split("-")[0])
        
        # ---------------------
        # lineup iterating rows calculation
        last_lineup = lineup_time_dict["lineup"][-1]
        if sorted(in_lineup) != sorted(last_lineup) and len(in_lineup) == 5:
            cached_time = lineup_time_dict["timecache"][-1]
            if lineup_time_dict["timecache"][-1] == "not_changed":
                enter_time = datetime.strptime("10:00", "%M:%S")
                enter_score = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0])
            else:
                enter_time = datetime.strptime(cached_time, "%M:%S")
                enter_score = int(lineup_time_dict["scorecache"][-1].split("-")[0])

            seconds = enter_time - datetime.strptime(row["Time"], "%M:%S")
            seconds = seconds.total_seconds()

            curr_score = int(row["Score"].split("-")[0])
            points = curr_score - enter_score

            lineup_time_dict["seconds"][-1] += seconds
            lineup_time_dict["points_conceded"][-1] += points

            lineup_time_dict["lineup"].append(sorted(in_lineup.copy()))
            for k in lineup_time_dict:
                if k != "lineup":
                    if "cache" in k:
                        lineup_time_dict[k].append("not_changed")
                    else:
                        lineup_time_dict[k].append(0)

            lineup_time_dict["timecache"].append(row["Time"])
            lineup_time_dict["scorecache"].append(row["Score"])
        # ---------------------
    # -------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------------------------------------
        # filling event_num_dict for meaesuring how many times each event occured
        if row["event"] not in event_num_dict:
            event_num_dict[row["event"]] = list(np.zeros(len(event_num_dict["player"])))
        
        if row["player"] not in event_num_dict["player"]:
            event_num_dict["player"].append(row["player"])
            for key in event_num_dict:
                if key != "player":
                    event_num_dict[key].append(0)
        
        player_index = event_num_dict["player"].index(row["player"])
        event_num_dict[row["event"]][player_index] += 1

        # ---------------------
        # lineup event calculations
        if len(lineup_event_dict["lineup"]) == 0:
            lineup_event_dict["lineup"].append(sorted(in_lineup))

        last_lineup = lineup_event_dict["lineup"][-1]
        new_lineup = sorted(in_lineup.copy())
        if row["event"] not in lineup_event_dict:
            lineup_event_dict[row["event"]] = list(np.zeros(len(lineup_event_dict["lineup"])))
        
        if new_lineup == last_lineup:
            lineup_event_dict[row["event"]][-1] += 1
        elif new_lineup != last_lineup and len(new_lineup) == 5:
            for key in list(lineup_event_dict.keys()):
                if key == "lineup":
                    lineup_event_dict[key].append(new_lineup)
                else:
                    lineup_event_dict[key].append(0)
        # ---------------------
# ------------------------------------------------------------------------------------------------------------------------------------------------------------

time_score_df = pd.DataFrame(quarter_dict)
lineup_time_score_df = pd.DataFrame(lineup_quarter_dict)

columns_ls = []
for i in range(1, 5):
    for sub in ["time", "score"]:
        columns_ls.append((f"quarter{i}", sub))

time_score_df.columns = pd.MultiIndex.from_tuples([("player", "player")] + columns_ls)
lineup_time_score_df.columns = pd.MultiIndex.from_tuples([("lineup", "lineup")] + columns_ls)

In [121]:
ls = [1, 2, 3]
ls1 = [4, 5, 6]

ls.append(ls1)
print(ls)

[1, 2, 3, [4, 5, 6]]


In [122]:
event_num_df = pd.DataFrame(event_num_dict)
lineup_event_df = pd.DataFrame(lineup_event_dict)
time_sum_ls = [(f"quarter{i}", "time") for i in range(1, 5)]
score_sum_ls = [(f"quarter{i}", "score") for i in range(1, 5)]
time_score_df[("total", "time")] = time_score_df[time_sum_ls].sum(axis=1)
time_score_df[("total", "score")] = time_score_df[score_sum_ls].sum(axis=1)
lineup_time_score_df[("total", "time")] = lineup_time_score_df[time_sum_ls].sum(axis=1)
lineup_time_score_df[("total", "score")] = lineup_time_score_df[score_sum_ls].sum(axis=1)

In [156]:
lineup_event_df.iloc[:-1] # starter lineup will be added automatically at end of the match so we ignore it cause no event will happen there(try proving it by removing .iloc)

,lineup,made layup,Assist,Turnover,defensive rebound,enters the game,goes to the bench,missed 3-pt. jump shot,Foul,made 3-pt. jump shot,Steal,made free throw,missed free throw,made jump shot,missed jump shot,offensive rebound,missed layup
0,"[LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]",2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"[LATIFF,WAZIR, RILEY,BEN, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]",0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"[OKADO,MARJOK, RILEY,BEN, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]",1.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0
3,"[BAILEY,ELLIOT, LATIFF,WAZIR, MEYER,EVAN, OKADO,MARJOK, SERAPH,REGINALD JEAN]",0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"[KNIGHT,JACOB, LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, VREEKEN,CONNOR]",1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0
5,"[KNIGHT,JACOB, LATIFF,WAZIR, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]",0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0
6,"[DOLMAN,LUKA, LATIFF,WAZIR, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]",1.0,1.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
7,"[BAILEY,ELLIOT, MEYER,EVAN, OKADO,MARJOK, RILEY,BEN, WARNHOLTZ,AIDEN]",0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
8,"[LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,"[MEYER,EVAN, OKADO,MARJOK, RILEY,BEN, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]",0.0,1.0,1.0,2.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [126]:
lineup_time_score_df

lineup  \
                                                                                   lineup   
0   [LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
1              [LATIFF,WAZIR, RILEY,BEN, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
2              [OKADO,MARJOK, RILEY,BEN, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
3           [BAILEY,ELLIOT, LATIFF,WAZIR, MEYER,EVAN, OKADO,MARJOK, SERAPH,REGINALD JEAN]   
4      [KNIGHT,JACOB, LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, VREEKEN,CONNOR]   
5           [KNIGHT,JACOB, LATIFF,WAZIR, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
6            [DOLMAN,LUKA, LATIFF,WAZIR, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
7                   [BAILEY,ELLIOT, MEYER,EVAN, OKADO,MARJOK, RILEY,BEN, WARNHOLTZ,AIDEN]   
8   [LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
9                  [MEYER,EVAN, OKADO,MARJOK, RILEY,BEN, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
10         [BAILEY,ELLIOT, LATIFF,WAZIR, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
11   [BAILEY,ELLIOT, LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, WARNHOLTZ,AIDEN]   
12             [OKADO,MARJOK, RILEY,BEN, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
13  [OKADO,MARJOK, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
14      [BAILEY,ELLIOT, KNIGHT,JACOB, LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT]   
15              [BAILEY,ELLIOT, LATIFF,WAZIR, RILEY,BEN, SHEPHARD,GRANT, WARNHOLTZ,AIDEN]   
16                 [BAILEY,ELLIOT, LATIFF,WAZIR, OKADO,MARJOK, RILEY,BEN, SAMAHA,GABRAEL]   
17                     [BAILEY,ELLIOT, LATIFF,WAZIR, MEYER,EVAN, OKADO,MARJOK, RILEY,BEN]   
18  [LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
19           [BAILEY,ELLIOT, LATIFF,WAZIR, OKADO,MARJOK, SHEPHARD,GRANT, WARNHOLTZ,AIDEN]   
20           [BAILEY,ELLIOT, LATIFF,WAZIR, OKADO,MARJOK, SAMAHA,GABRAEL, WARNHOLTZ,AIDEN]   
21         [BAILEY,ELLIOT, LATIFF,WAZIR, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
22              [BAILEY,ELLIOT, LATIFF,WAZIR, RILEY,BEN, SAMAHA,GABRAEL, WARNHOLTZ,AIDEN]   
23              [BAILEY,ELLIOT, LATIFF,WAZIR, RILEY,BEN, SHEPHARD,GRANT, WARNHOLTZ,AIDEN]   
24  [LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
25              [BAILEY,ELLIOT, LATIFF,WAZIR, RILEY,BEN, SHEPHARD,GRANT, WARNHOLTZ,AIDEN]   
26              [BAILEY,ELLIOT, LATIFF,WAZIR, RILEY,BEN, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
27             [LATIFF,WAZIR, RILEY,BEN, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
28              [BAILEY,ELLIOT, LATIFF,WAZIR, RILEY,BEN, SHEPHARD,GRANT, WARNHOLTZ,AIDEN]   
29         [BAILEY,ELLIOT, LATIFF,WAZIR, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   
30             [LATIFF,WAZIR, RILEY,BEN, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]   

   quarter1       quarter2       quarter3       quarter4        total        
       time score     time score     time score     time score   time score  
0      75.0   0.0      0.0   0.0      0.0   0.0      0.0   0.0   75.0   0.0  
1      83.0   2.0      0.0   0.0      0.0   0.0      0.0   0.0   83.0   2.0  
2      82.0   3.0      0.0   0.0      0.0   0.0      0.0   0.0   82.0   3.0  
3      79.0   0.0      0.0   0.0      0.0   0.0      0.0   0.0   79.0   0.0  
4      78.0   1.0      0.0   0.0      0.0   0.0      0.0   0.0   78.0   1.0  
5      75.0   0.0      0.0   0.0      0.0   0.0      0.0   0.0   75.0   0.0  
6      89.0   2.0      0.0   0.0      0.0   0.0      0.0   0.0   89.0   2.0  
7      78.0   0.0      0.0   0.0      0.0   0.0      0.0   0.0   78.0   0.0  
8       0.0   0.0      0.0   0.0      0.0   0.0      0.0   0.0    0.0   0.0  
9       0.0   0.0    105.0   2.0      0.0   0.0      0.0   0.0  105.0   2.0  
10      0.0   0.0     35.0   3.0      0.0   0.0      0.0   0.0   35.0   3.0  

In [127]:
event_num_df

,player,made layup,Assist,Turnover,defensive rebound,enters the game,goes to the bench,missed 3-pt. jump shot,Foul,made 3-pt. jump shot,Steal,made free throw,missed free throw,made jump shot,missed jump shot,offensive rebound,missed layup
0,"VREEKEN,CONNOR",4,0.0,1.0,5.0,6.0,9.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,1.0,2.0,2.0
1,"WARNHOLTZ,AIDEN",1,12.0,2.0,10.0,2.0,3.0,5.0,4.0,3.0,0.0,6.0,1.0,0.0,3.0,2.0,5.0
2,"SERAPH,REGINALD JEAN",0,1.0,1.0,0.0,4.0,8.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,"LATIFF,WAZIR",0,6.0,2.0,5.0,4.0,5.0,6.0,1.0,2.0,3.0,2.0,0.0,0.0,0.0,0.0,5.0
4,"SHEPHARD,GRANT",5,0.0,1.0,8.0,5.0,7.0,0.0,2.0,0.0,0.0,9.0,6.0,0.0,0.0,6.0,5.0
5,"RILEY,BEN",0,0.0,1.0,1.0,8.0,4.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0
6,"SAMAHA,GABRAEL",2,0.0,0.0,1.0,6.0,6.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0
7,"OKADO,MARJOK",0,1.0,2.0,3.0,7.0,5.0,1.0,5.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0
8,"BAILEY,ELLIOT",4,1.0,1.0,0.0,8.0,5.0,2.0,5.0,1.0,2.0,1.0,4.0,0.0,1.0,2.0,1.0
9,"MEYER,EVAN",0,0.0,0.0,1.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [154]:
for index, row in lineup_event_df.iterrows():
    if index < 31:
        time_lineup = lineup_time_score_df.iloc[index]["lineup"].to_list()[0]
    else:
        time_lineup = []
    event_lineup = lineup_event_df.iloc[index]["lineup"]
    if time_lineup != event_lineup:
        print(event_lineup, time_lineup, index)

['LATIFF,WAZIR', 'SERAPH,REGINALD JEAN', 'SHEPHARD,GRANT', 'VREEKEN,CONNOR', 'WARNHOLTZ,AIDEN'] [] 31


off possession = made and miss

def possession = Turnover, defensive rebound,	Foul, Steal

efficiency = point, rebound, assist, steal, block, -missed free throw, -turnover, -points conceded

What type of game was it? (Pre-season, Regular Season, or Playoffs)

In [166]:
scoring_values = {'made layup': 2, 'made free throw': 1, 'made jump shot': 2, 'made 3-pt. jump shot': 3}
final_columns = ['Player Name', 'PtScored', 'ptsconceded', 
                #  'off possession', 'def possession', 
                "efficiency",
                'minutes', 'home/visitor', 'opponent', 'date', 'game_type']
player_final_table = pd.DataFrame(columns=[final_columns])
for index, row in event_num_df.iterrows():
    
    points_scored = 0
    for key in scoring_values:
        score_value = scoring_values[key]
        points_scored += event_num_df.loc[index, key] * score_value

    points_conceded = float(time_score_df.loc[time_score_df[("player", "player")] == row["player"]][("total", "score")])

    efficiency = sum([points_scored, row["offensive rebound"], row["defensive rebound"], row["Assist"], row["Steal"]]) - \
    sum([row["missed free throw"], row["Turnover"], points_conceded])
    efficiency = float(efficiency)

    minutes = time_score_df.loc[time_score_df[("player", "player")] == row["player"]][("total", "time")] / 60
    minutes = "{:.2f}".format(minutes.to_list()[0])
    hv_df = player_event_df.loc[player_event_df["player"] == row["player"]].iloc[1]
    if pd.isna(hv_df['Home']) == False:
        hv = "Home"
    else:
        hv = "Visitor"
    
    opponent_df = player_event_df.loc[pd.isna(player_event_df[hv]) == True]
    opponent = opponent_df.iloc[1]["Home"] if hv == "Visitor" else opponent_df.iloc[1]["Visitor"]

    new_row = {"Player Name":[row["player"]], "PtScored":[points_scored],
               "ptsconceded":[points_conceded], "efficiency":[efficiency], "minutes":minutes, 
               "home/visitor":hv, "opponent":opponent}
    
    new_df = pd.DataFrame(new_row)
    player_final_table = pd.concat([player_final_table, new_df], ignore_index=True, axis=0)

player_final_table = player_final_table.reindex(columns=final_columns)

/var/folders/hx/k_zr_cdj5l348hsyxvym51kr0000gn/T/ipykernel_2956/1449936464.py:14: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  points_conceded = float(time_score_df.loc[time_score_df[("player", "player")] == row["player"]][("total", "score")])
/var/folders/hx/k_zr_cdj5l348hsyxvym51kr0000gn/T/ipykernel_2956/1449936464.py:14: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  points_conceded = float(time_score_df.loc[time_score_df[("player", "player")] == row["player"]][("total", "score")])


In [167]:
player_final_table

,Player Name,PtScored,ptsconceded,efficiency,minutes,home/visitor,opponent,date,game_type
0,"VREEKEN,CONNOR",11.0,32.0,-13.0,24.90,Visitor,Laurentian,NaN,NaN
1,"WARNHOLTZ,AIDEN",17.0,53.0,-15.0,34.92,Visitor,Laurentian,NaN,NaN
2,"SERAPH,REGINALD JEAN",0.0,18.0,-18.0,8.27,Visitor,Laurentian,NaN,NaN
3,"LATIFF,WAZIR",8.0,43.0,-23.0,29.88,Visitor,Laurentian,NaN,NaN
4,"SHEPHARD,GRANT",19.0,33.0,-7.0,19.25,Visitor,Laurentian,NaN,NaN
5,"RILEY,BEN",2.0,33.0,-31.0,23.23,Visitor,Laurentian,NaN,NaN
6,"SAMAHA,GABRAEL",6.0,20.0,-13.0,14.13,Visitor,Laurentian,NaN,NaN
7,"OKADO,MARJOK",5.0,20.0,-11.0,11.17,Visitor,Laurentian,NaN,NaN
8,"BAILEY,ELLIOT",12.0,28.0,-16.0,11.92,Visitor,Laurentian,NaN,NaN
9,"MEYER,EVAN",0.0,2.0,-1.0,4.87,Visitor,Laurentian,NaN,NaN


## Let's go to lineup

In [195]:
if "Player Name" in final_columns:
    final_columns.remove("Player Name")
lineup_final_columns = ["Lineup"].extend(final_columns)
lineup_final_table = pd.DataFrame(columns=[lineup_final_columns])
for index, row in lineup_event_df.iloc[:-1].iterrows():
    
    points_scored = 0
    for key in scoring_values:
        score_value = scoring_values[key]
        points_scored += lineup_event_df.loc[index, key] * score_value

    points_conceded = float(lineup_time_score_df[("total", "score")].iloc[index])

    efficiency = sum([points_scored, row["offensive rebound"], row["defensive rebound"], row["Assist"], row["Steal"]]) - \
    sum([row["missed free throw"], row["Turnover"], points_conceded])
    efficiency = float(efficiency)

    minutes = lineup_time_score_df[("total", "time")].iloc[index] / 60
    minutes = "{:.2f}".format(minutes)

    hv_df = player_event_df.loc[player_event_df["player"] == row["lineup"][0]].iloc[1]
    if pd.isna(hv_df['Home']) == False:
        hv = "Home"
    else:
        hv = "Visitor"

    opponent_df = player_event_df.loc[pd.isna(player_event_df[hv]) == True]
    opponent = opponent_df.iloc[1]["Home"] if hv == "Visitor" else opponent_df.iloc[1]["Visitor"]

    new_row = {"Lineup":[row["lineup"]], "PtScored":[points_scored],
               "ptsconceded":[points_conceded], "efficiency":[efficiency], 
               "minutes":minutes, "home/visitor":hv, "opponent":opponent}
    
    new_df = pd.DataFrame(new_row)
    lineup_final_table = pd.concat([lineup_final_table, new_df], ignore_index=True, axis=0)

lineup_final_table = lineup_final_table.reindex(columns=lineup_final_columns)

In [196]:
lineup_final_table

,None,Lineup,PtScored,ptsconceded,efficiency,minutes,home/visitor,opponent
0,NaN,"[LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]",4.0,0.0,6.0,1.25,Visitor,Laurentian
1,NaN,"[LATIFF,WAZIR, RILEY,BEN, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]",0.0,2.0,-1.0,1.38,Visitor,Laurentian
2,NaN,"[OKADO,MARJOK, RILEY,BEN, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]",9.0,3.0,9.0,1.37,Visitor,Laurentian
3,NaN,"[BAILEY,ELLIOT, LATIFF,WAZIR, MEYER,EVAN, OKADO,MARJOK, SERAPH,REGINALD JEAN]",3.0,0.0,5.0,1.32,Visitor,Laurentian
4,NaN,"[KNIGHT,JACOB, LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, VREEKEN,CONNOR]",4.0,1.0,5.0,1.30,Visitor,Laurentian
5,NaN,"[KNIGHT,JACOB, LATIFF,WAZIR, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]",3.0,0.0,7.0,1.25,Visitor,Laurentian
6,NaN,"[DOLMAN,LUKA, LATIFF,WAZIR, SAMAHA,GABRAEL, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]",2.0,2.0,3.0,1.48,Visitor,Laurentian
7,NaN,"[BAILEY,ELLIOT, MEYER,EVAN, OKADO,MARJOK, RILEY,BEN, WARNHOLTZ,AIDEN]",4.0,0.0,6.0,1.30,Visitor,Laurentian
8,NaN,"[LATIFF,WAZIR, SERAPH,REGINALD JEAN, SHEPHARD,GRANT, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]",0.0,0.0,0.0,0.00,Visitor,Laurentian
9,NaN,"[MEYER,EVAN, OKADO,MARJOK, RILEY,BEN, VREEKEN,CONNOR, WARNHOLTZ,AIDEN]",3.0,2.0,3.0,1.75,Visitor,Laurentian


In [37]:
import pandas as pd

# Create a sample dataframe
data = {'A': [1, 2, 3], 'B': [4, 5, 6], "C":[4, 5, 6]}
df = pd.DataFrame(data)

# Create multi-level columns
df.columns = pd.MultiIndex.from_tuples([('Column 1', 'time'), ('Column 1', 'score'), ('Column 2', 'time')])

# Display the dataframe
print(df)

  Column 1       Column 2
      time score     time
0        1     4        4
1        2     5        5
2        3     6        6
